In [5]:
import preprocessing 
import json
import sys
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import numpy as np
import numpy.matlib as npm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [6]:
df = pd.read_csv("./PENGUINS_20230601_to_20230630_events_simplified.tsv", sep = "\t")
df=df.sort_values(["session_id","index"])
df.loc[:,"game_state"]=df["game_state"].apply(json.loads)
df.loc[:,"event_data"]=df["event_data"].apply(json.loads)
df.head()



,Unnamed: 0,session_id,app_id,timestamp,event_name,event_data,event_source,app_version,app_branch,log_version,offset,user_id,user_data,game_state,index
21,21,23060111411894318,PENGUINS,2023-06-01T16:41:18.767000+00:00,application_start,{'http_user_agent': 'UnityPlayer/2021.3.17f1 (...,GAME,102,NaN,7,"-1 day, 23:59:42",NaN,{},"{'posX': 0, 'posY': 0, 'posZ': 0, 'rotW': 1, '...",0
26,26,23060111411894318,PENGUINS,2023-06-01T16:41:19.394000+00:00,headset_on,{'http_user_agent': 'UnityPlayer/2021.3.17f1 (...,GAME,102,NaN,7,"-1 day, 23:59:42",NaN,{},"{'posX': -2.765, 'posY': 2.076, 'posZ': 57.012...",1
31,31,23060111411894318,PENGUINS,2023-06-01T16:41:19.394000+00:00,open_menu,{'http_user_agent': 'UnityPlayer/2021.3.17f1 (...,GAME,102,NaN,7,"-1 day, 23:59:42",NaN,{},"{'posX': -2.765, 'posY': 2.68, 'posZ': 57.012,...",2
5,5,23060111411894318,PENGUINS,2023-06-01T16:41:20.423000+00:00,viewport_data,"{'gaze_data_package': '[{""pos"":[-2.545,3.854,5...",GAME,102,NaN,7,"-1 day, 23:59:42",NaN,{},"{'posX': -2.573, 'posY': 2.473, 'posZ': 56.364...",3
1,1,23060111411894318,PENGUINS,2023-06-01T16:41:20.424000+00:00,left_hand_data,{'http_user_agent': 'UnityPlayer/2021.3.17f1 (...,GAME,102,NaN,7,"-1 day, 23:59:42",NaN,{},"{'posX': -2.573, 'posY': 2.473, 'posZ': 56.364...",4


In [7]:
#get timestamp from game_state

df['timesincelaunch'] = df['game_state'].apply(lambda x: x.get('seconds_from_launch', 0))
print(df.columns)

df_copy = df[['session_id','timestamp','event_name','event_data','game_state',"timesincelaunch"]]
df_copy['timestamp'] = pd.to_datetime(df_copy['timestamp'],format="mixed",yearfirst=True)

Index(['Unnamed: 0', 'session_id', 'app_id', 'timestamp', 'event_name',
       'event_data', 'event_source', 'app_version', 'app_branch',
       'log_version', 'offset', 'user_id', 'user_data', 'game_state', 'index',
       'timesincelaunch'],
      dtype='object')


/tmp/ipykernel_2268/417955552.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['timestamp'] = pd.to_datetime(df_copy['timestamp'],format="mixed",yearfirst=True)


In [8]:
gaze_package_lst = preprocessing.process_data_package("viewport_data","gaze_data_package",df_copy)
gaze_package_lst.head()   
rt_hand_package_lst = preprocessing.process_data_package("right_hand_data","right_hand_data_package",df_copy)
rt_hand_package_lst.head()
lft_hand_package_lst = preprocessing.process_data_package("left_hand_data","left_hand_data_package",df_copy)
lft_hand_package_lst.head()

,session_id,timestamp,event_name,event_data,game_state,timesincelaunch,headset_on_counter,position,rotation,timesincelaunch_diff,timesincelaunch_initial,num_items,timesincelaunch_diff_split,sequence,player_id
0,23060111411894318,2023-06-01 16:41:20.424000+00:00,left_hand_data,"[{'pos': [-2.45, 3.876, 56.373], 'rot': [0.177...","{'posX': -2.573, 'posY': 2.473, 'posZ': 56.364...",0.038607,1,"[-2.45, 3.876, 56.373]","[0.177, 0.756, 0.626, 0.07]",1.081,0.0,28,0.038607,0,23060111411894318-1
1,23060111411894318,2023-06-01 16:41:20.424000+00:00,left_hand_data,"[{'pos': [-2.45, 3.876, 56.373], 'rot': [0.177...","{'posX': -2.573, 'posY': 2.473, 'posZ': 56.364...",0.077214,1,"[-2.448, 3.67, 56.373]","[0.163, 0.768, 0.616, 0.064]",1.081,0.0,28,0.038607,1,23060111411894318-1
2,23060111411894318,2023-06-01 16:41:20.424000+00:00,left_hand_data,"[{'pos': [-2.45, 3.876, 56.373], 'rot': [0.177...","{'posX': -2.573, 'posY': 2.473, 'posZ': 56.364...",0.115821,1,"[-2.46, 3.671, 56.385]","[0.123, 0.77, 0.621, 0.075]",1.081,0.0,28,0.038607,2,23060111411894318-1
3,23060111411894318,2023-06-01 16:41:20.424000+00:00,left_hand_data,"[{'pos': [-2.45, 3.876, 56.373], 'rot': [0.177...","{'posX': -2.573, 'posY': 2.473, 'posZ': 56.364...",0.154429,1,"[-2.466, 2.499, 56.392]","[0.111, 0.758, 0.637, 0.083]",1.081,0.0,28,0.038607,3,23060111411894318-1
4,23060111411894318,2023-06-01 16:41:20.424000+00:00,left_hand_data,"[{'pos': [-2.45, 3.876, 56.373], 'rot': [0.177...","{'posX': -2.573, 'posY': 2.473, 'posZ': 56.364...",0.193036,1,"[-2.47, 2.498, 56.4]","[0.105, 0.741, 0.658, 0.09]",1.081,0.0,28,0.038607,4,23060111411894318-1


In [9]:
gaze_package_lst_splt = preprocessing.split_dataframe_by_player_id(gaze_package_lst,"player_id")
lft_hand_package_lst_splt = preprocessing.split_dataframe_by_player_id(lft_hand_package_lst,"player_id")
rt_hand_package_lst_splt = preprocessing.split_dataframe_by_player_id(rt_hand_package_lst,"player_id")


for i in range(len(gaze_package_lst_splt)):
  gaze_package_lst_splt[i] ['game_state_pos'] = gaze_package_lst_splt[i] ['game_state'].apply(lambda x: [x.get('posX', 0), x.get('posY', 0), x.get('posZ', 0)])
for i in range(len(lft_hand_package_lst_splt)):
  lft_hand_package_lst_splt[i] ['game_state_pos'] = lft_hand_package_lst_splt[i] ['game_state'].apply(lambda x: [x.get('posX', 0), x.get('posY', 0), x.get('posZ', 0)])
for i in range(len(rt_hand_package_lst_splt)):
  rt_hand_package_lst_splt[i] ['game_state_pos'] = rt_hand_package_lst_splt[i] ['game_state'].apply(lambda x: [x.get('posX', 0), x.get('posY', 0), x.get('posZ', 0)])


for i in range(len(gaze_package_lst_splt)):
  gaze_package_lst_splt[i] ['game_state_rot'] = gaze_package_lst_splt[i] ['game_state'].apply(lambda x: [x.get('rotW', 0), x.get('rotX', 0), x.get('rotY', 0), x.get('rotZ', 0)])

for i in range(len(lft_hand_package_lst_splt)):
  lft_hand_package_lst_splt[i] ['game_state_rot'] = lft_hand_package_lst_splt[i] ['game_state'].apply(lambda x: [x.get('rotW', 0), x.get('rotX', 0), x.get('rotY', 0), x.get('rotZ', 0)])
  
for i in range(len(rt_hand_package_lst_splt)):
  rt_hand_package_lst_splt[i] ['game_state_rot'] = rt_hand_package_lst_splt[i] ['game_state'].apply(lambda x: [x.get('rotW', 0), x.get('rotX', 0), x.get('rotY', 0), x.get('rotZ', 0)])


In [10]:
split_gaze_packages = preprocessing.average_dataframes_by_rows(gaze_package_lst_splt,1)
split_lft_hand_packages =preprocessing.average_dataframes_by_rows(lft_hand_package_lst_splt,1)
split_rt_hand_packages=preprocessing.average_dataframes_by_rows(rt_hand_package_lst_splt,1)

/workspaces/OGDUtils/src/OGDUtils/VRTeX/preprocessing.py:117: FutureWarning: The provided callable <function mean at 0x7f20081209d0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  avg_df = df_grouped.agg({
/workspaces/OGDUtils/src/OGDUtils/VRTeX/preprocessing.py:117: FutureWarning: The provided callable <function mean at 0x7f20081209d0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  avg_df = df_grouped.agg({
/workspaces/OGDUtils/src/OGDUtils/VRTeX/preprocessing.py:117: FutureWarning: The provided callable <function mean at 0x7f20081209d0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  avg_df = df_grouped.agg({
/

In [ ]:
# split_gaze_packages = sample_dataframes_by_rows(gaze_package_lst_splt,5)
# split_lft_hand_packages =sample_dataframes_by_rows(lft_hand_package_lst_splt,5)
# split_rt_hand_packages=sample_dataframes_by_rows(rt_hand_package_lst_splt,5)

In [12]:
split_gaze_packages = preprocessing.norm_average(split_gaze_packages)
split_lft_hand_packages = preprocessing.norm_average(split_lft_hand_packages)
split_rt_hand_packages = preprocessing.norm_average(split_rt_hand_packages)

In [13]:
split_gaze_packages = preprocessing.normalize_3d(split_gaze_packages)
split_lft_hand_packages = preprocessing.normalize_3d(split_lft_hand_packages)
split_rt_hand_packages = preprocessing.normalize_3d(split_rt_hand_packages)

In [14]:
split_gaze_packages[0]

,position,rotation,timesincelaunch,game_state_pos,event_name,rotation_x,average_view_pos,norm_average_view_pos,3d_quat,3d_normalized
0,"[-2.545, 3.854, 56.294]","[0.07596506410392102, 0.9175780111499937, -0.1...",0.038607,"[-2.573, 2.473, 56.364]",viewport_data,0.917578,19.201000,1.0,"[-0.7089637533469213, 0.055708747951884297, -0...","[-0.7089637533469212, 0.05570874795188428, -0...."
1,"[-2.544, 3.644, 56.304]","[0.0709711915431635, 0.9066319821077369, -0.11...",0.077214,"[-2.573, 2.473, 56.364]",viewport_data,0.906632,19.134667,1.0,"[-0.6685851088915803, 0.03973973133815342, -0....","[-0.6685851088915802, 0.039739731338153415, -0..."
2,"[-2.547, 3.647, 56.313]","[-0.06701048796211334, -0.9021411961466601, 0....",0.115821,"[-2.573, 2.473, 56.364]",viewport_data,-0.902141,19.137667,1.0,"[-0.6497743793807461, 0.03414868853951289, -0....","[-0.6497743793807462, 0.034148688539512896, -0..."
3,"[-2.547, 2.476, 56.314]","[0.067999932000102, 0.9029990970013545, -0.087...",0.154429,"[-2.573, 2.473, 56.364]",viewport_data,0.902999,18.747667,1.0,"[-0.6463027073945853, 0.049767900464199086, -0...","[-0.6463027073945853, 0.049767900464199086, -0..."
4,"[-2.547, 2.478, 56.316]","[-0.06800639290141326, -0.9020847999569812, 0....",0.193036,"[-2.573, 2.473, 56.364]",viewport_data,-0.902085,18.749000,1.0,"[-0.6378839221773692, 0.06220369429452743, -0....","[-0.6378839221773692, 0.06220369429452743, -0...."
...,...,...,...,...,...,...,...,...,...,...
630,"[-2.519, 2.486, 56.318]","[-0.05698749261790492, -0.3069326356788916, -0...",19.040667,"[-2.523, 2.492, 56.336]",viewport_data,-0.306933,18.761667,1.0,"[0.8107031013385122, 0.07486513420608354, -0.5...","[0.8107031013385123, 0.07486513420608355, -0.5..."
631,"[-2.52, 2.487, 56.322]","[-0.05700527323167219, -0.30902858646643333, -...",19.0685,"[-2.523, 2.492, 56.336]",viewport_data,-0.309029,18.763000,1.0,"[0.8080344863799803, 0.07699624430519648, -0.5...","[0.8080344863799803, 0.07699624430519648, -0.5..."
632,"[-2.522, 2.489, 56.327]","[0.05601274434917408, 0.31207100423111295, 0.0...",19.096333,"[-2.523, 2.492, 56.336]",viewport_data,0.312071,18.764667,1.0,"[0.8045010479768292, 0.0710003051388382, -0.58...","[0.8045010479768292, 0.0710003051388382, -0.58..."
633,"[-2.523, 2.491, 56.331]","[0.055980634050348456, 0.31389141235373974, 0....",19.124167,"[-2.523, 2.492, 56.336]",viewport_data,0.313891,18.766333,1.0,"[0.8024327165601401, 0.06545870257781616, -0.5...","[0.8024327165601403, 0.06545870257781618, -0.5..."
